In [30]:
import pyspark
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, randn
import numpy as np
import pandas as pd
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [3]:
spark = SparkSession.builder.appName("Salary Prediction").getOrCreate()

24/11/03 13:20:16 WARN Utils: Your hostname, sathvik-HP-EliteBook-x360-1030-G2 resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp58s0)
24/11/03 13:20:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 13:20:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
df = spark.read.csv("../Data/adult.csv", header=False, inferSchema=True)
column_names = ["age", "workclass", "fnlwgt", "education", "education_num", 
                "marital_status", "occupation", "relationship", "race", 
                "sex", "capital_gain", "capital_loss", "hours_per_week", 
                "native_country", "income"]

df = df.toDF(*column_names)
df.show(5)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

In [24]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)



In [28]:
df.describe().show(5)

24/11/03 13:40:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------+
|summary|               age|   workclass|            fnlwgt|    education|    education_num|marital_status|       occupation|relationship|               race|    sex|      capital_gain|    capital_loss|    hours_per_week|native_country|income|
+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------+
|  count|             32561|       32561|             32561|        32561|            32561|         32561|            32561|       32561|              32561|  32561|             32561|           32561|             32561|         32561| 32561|
|   mean| 38.58164675532

In [26]:
print(f"The number of observations in the dataframe: {df.count()}")
print(f"Number of features in the dataset: {len(df.columns)}")
print("Features: ")
print(df.columns)

The number of observations in the dataframe: 32561
Number of features in the dataset: 15
Features: 
['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']


In [27]:
df_clean = df.dropna()

In [31]:
categorical_columns = ["workclass", "education", "marital_status", "occupation", 
                "relationship", "race", "sex", "native_country", "income"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in categorical_columns]

assembler = VectorAssembler(inputCols=
                            ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week'] + 
                            [col + "_index" for col in categorical_columns],
              outputCol="features")

pipeline = Pipeline(stages=indexers + [assembler])
df_prepared = pipeline.fit(df_clean).transform(df_clean)

In [34]:
df_prepared.select("features", "income_index").show(5)

+--------------------+------------+
|            features|income_index|
+--------------------+------------+
|[39.0,77516.0,13....|         0.0|
|(15,[0,1,2,5,6,7,...|         0.0|
|(15,[0,1,2,5,8,9,...|         0.0|
|(15,[0,1,2,5,7,9,...|         0.0|
|[28.0,338409.0,13...|         0.0|
+--------------------+------------+
only showing top 5 rows



In [36]:
train_data, test_data = df_prepared.randomSplit([0.8, 0.2], seed=42)

dt = DecisionTreeClassifier(labelCol="income_index", featuresCol="features", maxBins=45)
dt_model = dt.fit(train_data)

predictions = dt_model.transform(test_data)

predictions.select("features", "income_index", "prediction").show(5)

+--------------------+------------+----------+
|            features|income_index|prediction|
+--------------------+------------+----------+
|[17.0,41643.0,7.0...|         0.0|       0.0|
|[17.0,64785.0,6.0...|         0.0|       0.0|
|[17.0,80077.0,7.0...|         0.0|       0.0|
|[17.0,104025.0,7....|         0.0|       0.0|
|[17.0,139183.0,6....|         0.0|       0.0|
+--------------------+------------+----------+
only showing top 5 rows



In [40]:
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="income_index", predictionCol="prediction", metricName="accuracy")
accuracy = accuracy_evaluator.evaluate(predictions)

precision_evaluator = MulticlassClassificationEvaluator(labelCol="income_index", predictionCol="prediction", metricName="precisionByLabel")
precision = precision_evaluator.evaluate(predictions)

recall_evaluator = MulticlassClassificationEvaluator(labelCol="income_index", predictionCol="prediction", metricName="recallByLabel")
recall = recall_evaluator.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
